# Introduction Pour Semantic Search et différence avec Keyword Search

# Step 1: Prepare documents

In [1]:
from tqdm.auto import tqdm
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

d:\Mon Drive\Cours mostafa\Formation LLM\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

# Step 2: Create Embeddings using Pretrained Models

In [3]:
# Import embedding model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

d:\Mon Drive\Cours mostafa\Formation LLM\venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
model.encode("This is a simple sentence") # We get back a vector 
len(model.encode("This is a simple sentence") ) # len of this vector using this model

768

In [5]:
#created the dense vector using the pre-trained model
operations = []
for doc in tqdm(documents):
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

100%|██████████| 948/948 [03:51<00:00,  4.10it/s]


# Step 3: Setup ElasticSearch connection

In [6]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '872842a38427', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'jirvhP_FQgSm5Qsaatj-3Q', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Step 4: Create Mappings and Index

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"}, # 768 is len of embedding model used & we have other similarity
        }
    }
}

In [8]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

# Step 5: Add documents into index

In [9]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

# Step 6: Create end user query
<h4>A savoir que la diff entre Semantic search et keyword search, au lieu de passer query directement dans la fct search on passe embaddung vector</h4>

In [10]:
search_term = "windows or mac?"
# Embending the search term
vector_search_term = model.encode(search_term)

In [11]:
# à optimiser
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5, # 5 nearest neighbors documents
    "num_candidates": 10000,
}

In [12]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"] # when _score is close to 1 it indicates that it's a very good much but to 0 indicates very bad much

[{'_index': 'course-questions',
  '_id': 'AUG-wpIBkkugYTEuW8gz',
  '_score': 0.71479195,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': '1UG-wpIBkkugYTEuacnW',
  '_score': 0.60555583,
  '_source': {'question': "The answer I get for one of the homework questions doesn't match any of the options. What should I do?",
   'course': 'machine-learning-zoomcamp',
   'section': '2. Machine Learning for Regression',
   'text': 'That’s normal. We all have different environments: our computers have different versions of OS and different versions of libraries — even different versions of Python.\nIf it’s the case, just select the option that’s closest to your answer'}},
 {'_index': 'course-questions',
  '

# Step 7 : Perform Keyword search with Semantic Search (Hybrid/ Advenced Search)

In [16]:
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term, 
    "k": 5, # 5 nearest neighbors documents
    "num_candidates": 10000, 
}

respone = es_client.search(
    index = index_name,
    query={
        "match":{
            "course":"data-engineering-zoomcamp"
        },
    },
    knn = knn_query,
    size=5,
    explain=True
)

In [17]:
respone['hits']['hits']

[{'_shard': '[course-questions][0]',
  '_node': 'xaQaTesqT2SQ4sPTOxWQww',
  '_index': 'course-questions',
  '_id': 'AUG-wpIBkkugYTEuW8gz',
  '_score': 1.493706,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.026965467259287834,
    -0.0006261487724259496,
    -0.016629496589303017,
    0.05285152047872543,
    0.054765284061431885,
    -0.031339891254901886,
    0.029942581430077553,
    -0.04808562994003296,
    0.04467552527785301,
    0.0058394744992256165,
    0.016233038157224655,
    0.012001179158687592,
    -0.03122226521372795,
    0.016600513830780983,
    -0.04886903986334801,
    -0.06496306508779526,
    0.046434223651885986,
    -0.009297751821577549,
    -0.0642528384923935,
    -0.0137327024713158